# Traffic Sign Recognition using Histogram of Oriented Gradient and Convolutional Neural Network
This notebook will showcase the training process of the traffic sign recognizer using neural network. Before you click Run All, it is important to check that the following packages were installed on your host machine.
  1. opencv-python
  2. tensorflow
  3. numpy
  4. scikit-image
  5. scipy

## Move your dataset to the src folder
Note that you should rename the files of a type as "XXXX_" where XXXX is the label of the image,
the label should not be more than 4 characters!


## Data Acqusition 
Loading the dataset along with the respective labels
If the dataset has one __80 speed limit__ sign and another __stop__ sign,
the labels will be ['80', 'Stop']
__Note__: The size of data and label should be the same, report to us if you find the size was different.

In [ ]:
import common as cm

cm.files_rename('src')
data, labels = cm.load_data('src')
print('Size of Data:', len(data))
print('Size of Label:', len(labels))

## Preprocessing Stage
Every image in the dataset will undergo the preprocessing stage which included the following operations:
  1. Histogram Normalization - to enhance the constrast and detail of an image
  2. Hough Transform - to detect the largest circle in the image and return the content within the circle, it was set to accept not so round shape as well.
  3. HOG feature descriptor - to extract the feature from the images, features as in the change in gradient.
  4. Reshaping - reshape the image to (*image.shape, 1) for the classifier
You may notice the data size was reduced to a lower number.
This is because the preprocessor will filter out those photos in awful quality, or when the Hough transform failed to detect round shape on the image.

In [ ]:
import numpy as np
from parallel import preprocess_image_parallel as preprocess

dataset, label = preprocess(data, labels) # dataset
data_size = len(dataset)
data_type = type(dataset)
print(f'''
Data size: {data_size}
Data type: {data_type}
''')

The following code shows the sizes of each type of traffic signs.
Depending on the traffic signs dataset you downloaded, the sizes of each type of traffic signs can be different.
The counting was presented as the following format:
(name_of_sign, size_of_sign)

In [ ]:
import collections

counter = collections.Counter(label)
counter.most_common()

## Preprocessing Stage 2
It is required to convert the image dataset to numpy array for the classifier to work with.
For the label, we first converted them into numeric form ('80' -> 0, 'SP' -> 1, etc.)
Then, the label was categorized into the CNN preferred format which is:
say there is 5 types of sign, and the first sign is __80 speed limit sign__ which is the index 0 after the conversion will become
  [1, 0, 0, 0, 0]
where the remaining 4 zeros are reserved for the other 4 signs.

In [ ]:
from tensorflow.keras.utils import to_categorical
from json import dumps

X = np.array(dataset)

print('X shape:', X.shape)

label_set = set(label)
classes = { val: key for (key, val) in enumerate(label_set)}
# cache the label types for testing purpose
output_text = dumps(classes)
output = open('classes.txt', 'w')
output.write(output_text)
output.close()
print(classes)

Y = np.fromiter([classes[y] for y in label], dtype=np.int)
Y = to_categorical(Y)
dense = len(Y[0])

## Data splitting
We need to separate part of the data for testing purpose.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape

## Contruct the layers of our CNN model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

# initiate model
model = Sequential()

# add model layers
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(512, 256, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(Flatten())
model.add(Dense(dense, activation='softmax'))
model.summary()

In [ ]:
# compile model using accuracy to measure the performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

## Never forget to save the model!

In [ ]:
model.save('saved_models/my_model')